In [10]:
import torch
import numpy as np
from tqdm import tqdm
import os
import torchvision
from PIL import Image
import open_clip


/Users/pshishodia/projects/Fashion/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
device = "mps" if torch.cuda.is_available() else "cpu"
torch.set_grad_enabled(False)

In [11]:
%%time
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
model = model.to(device)

/Users/pshishodia/projects/Fashion/venv/lib/python3.12/site-packages/open_clip/factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoi

CPU times: user 4.21 s, sys: 1.54 s, total: 5.75 s
Wall time: 22 s


In [14]:
all_images = os.listdir('assets-224')

In [15]:
INPUT_DIR = 'assets-224'

torch.from_numpy(np.array(Image.open(INPUT_DIR + '/' + all_images[0]))).shape

torch.Size([224, 224, 3])

In [25]:
INPUT_DIR = 'assets-224'
batch_size=500

all_images_features = []
for i in tqdm(range(0, len(all_images), batch_size)):
    batch_images = all_images[i:i+batch_size]
    batch_images = [preprocess(Image.open(INPUT_DIR + '/' + img_path)).unsqueeze(0) for img_path in batch_images]
    batch_images = torch.cat(batch_images, 0).to(device)

    batch_features = model.encode_image(batch_images)
    
    all_images_features.extend(batch_features)

100%|██████████| 615/615 [1:02:26<00:00,  6.09s/it]


In [26]:
embeddings_tensor = torch.stack(all_images_features)
torch.save(embeddings_tensor, 'image_embeddings_before.pt')

In [27]:
from tqdm import tqdm
import pandas as pd

## create img_path -> ix
image_url_to_idx = {}
for i, img_path in enumerate(all_images):
    image_url_to_idx[img_path] = i

def get_file_path_from_url(url):
    return '-'.join(url.split('/')[5:])

df = pd.read_csv('products_minimal.csv')

embedding_indexes = []
for _, row in tqdm(df.iterrows()):
    img_path = get_file_path_from_url(row['searchImage'])
    if img_path not in image_url_to_idx:
       print(f'img_path : {img_path}, index : {row['index']}')
       
    embedding_indexes.append(image_url_to_idx[img_path])

307184it [00:03, 91430.14it/s]


In [29]:
image_features_post = [all_images_features[ix] for ix in embedding_indexes]
embeddings_tensor = torch.stack(image_features_post)
torch.save(embeddings_tensor, 'image_embeddings_kaggle.pt')

In [5]:
## Old code ... do not edit. 
def save_all_images_as_tensor():
    batch_size = 500
    all_images = os.listdir(IMAGE_DIR)
    image_shape = torch.from_numpy(np.array(Image.open(IMAGE_DIR + '/' + all_images[0]))).shape
    all_images_tensors = torch.empty((len(all_images),) + image_shape, dtype=torch.uint8)

    for i in tqdm(range(0, len(all_images), batch_size)):
        batch_images = all_images[i:i+batch_size]

        batch_images = [torch.from_numpy(np.array(Image.open(IMAGE_DIR + '/' + img_path).convert('RGB'))) for img_path in batch_images]

        all_images_tensors[i:i+batch_size] = torch.stack(batch_images)

    print(all_images_tensors.shape)
    return all_images_tensors

In [7]:
normalize=torchvision.transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))